In [ ]:
pip install tensorflow

In [ ]:
pip install opencv-python


In [ ]:
pip install scikit-learn


# 모델 새로 생성

In [11]:
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import json

# JSON 파일 경로
json_file_path = 'C:/upload/IMG_OCR_53_4PO_09451.json'

# JSON 파일 읽고 지정된 범위의 데이터 추출
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    json_lines = json_file.readlines()[29:-1]

# JSON 데이터 파싱 및 "bbox" 부분 제거
json_data = ''.join(line.strip().replace('"bbox": ', '') for line in json_lines)
data = json.loads(json_data)

dict_list = data

# 이미지 로드 및 전처리 함수 정의
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (128, 128))
    return image

# 이미지 데이터와 레이블 준비
image = cv2.imread("C:/upload/image1.png")

X = []
y = []
for dict in dict_list:
    x1 = min(dict["x"])
    x2 = max(dict["x"])
    y1 = min(dict["y"])
    y2 = max(dict["y"])
    box_image = image[y1:y2, x1:x2]
    box_image = cv2.cvtColor(box_image, cv2.COLOR_BGR2GRAY)
    box_image = cv2.resize(box_image, (128, 128))
    X.append(box_image)
    y.append(dict["data"])

X = np.array(X)
y = np.array(y)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
y_onehot = to_categorical(y_encoded, num_classes=len(label_encoder.classes_))


# 데이터 확장을 위한 이미지 변환 함수 정의
import cv2
import random
import numpy as np

def augment_image(image):
    # 이미지 변환 방식 무작위 선택
    transform_type = random.choice(["rotate", "brightness", "color", "stretch", "noise"])

    if transform_type == "rotate":
        angle = random.randint(-30, 30)
        rows, cols = image.shape
        rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        transformed_image = cv2.warpAffine(image, rotation_matrix, (cols, rows))
    elif transform_type == "brightness":
        brightness_factor = random.uniform(0.7, 1.3)
        transformed_image = cv2.convertScaleAbs(image, alpha=brightness_factor, beta=0)
    elif transform_type == "color":
        color_factor = random.uniform(0.5, 1.5)
        transformed_image = image * color_factor
        transformed_image = np.clip(transformed_image, 0, 255).astype(np.uint8)
    elif transform_type == "stretch":
        stretch_factor_x = random.uniform(0.8, 1.2)
        stretch_factor_y = random.uniform(0.8, 1.2)
        rows, cols = image.shape
        stretched_image = cv2.resize(image, (int(cols * stretch_factor_x), int(rows * stretch_factor_y)))
        transformed_image = cv2.resize(stretched_image, (128, 128))
    elif transform_type == "noise":
        gaussian_noise = np.zeros((image.shape[0], image.shape[1], 1), dtype=np.uint8)
        cv2.randn(gaussian_noise, 128, 12)
        noisy_image = cv2.add(image, gaussian_noise)
        transformed_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

    return transformed_image



augmented_X = []
augmented_y_onehot = []

for i in range(X.shape[0]):
    original_image = X[i]
    augmented_X.append(original_image)
    augmented_y_onehot.append(y_onehot[i])

    # 이미지 확장을 위해 변환된 이미지 추가
    for _ in range(5):  # 30개의 변환된 이미지 추가
        transformed_image = augment_image(original_image)
        sized_transformed_image = cv2.resize(transformed_image, (128, 128))
        augmented_X.append(sized_transformed_image)
        augmented_y_onehot.append(y_onehot[i])

# 데이터 확장 후 배열로 변환
augmented_X = np.array(augmented_X)
augmented_y_onehot = np.array(augmented_y_onehot)

# 훈련 및 검증 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(augmented_X, augmented_y_onehot, test_size=0.2, random_state=42)

# 모델 생성
model = keras.Sequential([
    layers.Input(shape=(128, 128, 1)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.Dense(len(label_encoder.classes_), activation='softmax')
])

model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 조기 종료 콜백 정의
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# 모델 훈련
model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val), callbacks=[early_stopping])

# 새로운 이미지 예측 함수 정의
def predict_new_image(image_path, model, label_encoder):
    new_image = load_and_preprocess_image(image_path)
    prediction = model.predict(np.array([new_image]))
    predicted_class = np.argmax(prediction)
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    return predicted_label

# 새로운 이미지 예측
new_image_path = 'C:/upload/image3.png'
predicted_label = predict_new_image(new_image_path, model, label_encoder)
print("예측된 레이블:", predicted_label)


# 모델 저장
model_filepath = 'C:/upload/handwriting_model.h5'
label_encoder_filepath = 'C:/upload/handwriting_label_encoder.pkl'

import pickle
# 모델 저장 함수 정의
def save_model(model, model_filepath, label_encoder_filepath):
    model.save(model_filepath)

    with open(label_encoder_filepath, 'wb') as f:
        pickle.dump(label_encoder, f)

        
save_model(model, model_filepath, label_encoder_filepath)

Epoch 1/5
9/9 [==============================] - 4s 356ms/step - loss: 5.7174 - accuracy: 0.0576 - val_loss: 6.1429 - val_accuracy: 0.0000e+00
Epoch 2/5
9/9 [==============================] - 3s 349ms/step - loss: 4.0123 - accuracy: 0.1331 - val_loss: 4.9569 - val_accuracy: 0.0000e+00
Epoch 3/5
9/9 [==============================] - 3s 344ms/step - loss: 3.2498 - accuracy: 0.2662 - val_loss: 4.6950 - val_accuracy: 0.0143
Epoch 4/5
9/9 [==============================] - 3s 332ms/step - loss: 2.6425 - accuracy: 0.4173 - val_loss: 4.5841 - val_accuracy: 0.0000e+00
Epoch 5/5
1/1 [==============================] - 0s 82ms/step
예측된 레이블: 유앙페


In [17]:
new_image_path = 'C:/upload/image6.png'
predicted_label = predict_new_image(new_image_path, model, label_encoder)
print("예측된 레이블:", predicted_label)

NameError: name 'predict_new_image' is not defined

# 기존 모델에 추가

In [14]:
model_filepath = 'C:/upload/handwriting_model.h5'
label_encoder_filepath = 'C:/upload/handwriting_label_encoder.pkl'

In [15]:
def load_model(model_filepath, label_encoder_filepath):
    loaded_model = keras.models.load_model(model_filepath)

    with open(label_encoder_filepath, 'rb') as f:
        loaded_label_encoder = pickle.load(f)

    return loaded_model, loaded_label_encoder

In [16]:
loaded_model, loaded_label_encoder = load_model(model_filepath, label_encoder_filepath)

In [17]:
import cv2
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import json

# JSON 파일 경로
json_file_path = 'C:/upload/IMG_OCR_53_4PO_09451.json'

# JSON 파일 읽고 지정된 범위의 데이터 추출
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    json_lines = json_file.readlines()[29:-1]

# JSON 데이터 파싱 및 "bbox" 부분 제거
json_data = ''.join(line.strip().replace('"bbox": ', '') for line in json_lines)
data = json.loads(json_data)

dict_list = data

# 이미지 로드 및 전처리 함수 정의
def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (128, 128))
    return image

# 이미지 데이터와 레이블 준비
image = cv2.imread("C:/upload/image1.png")

X = []
y = []
for dict in dict_list:
    x1 = min(dict["x"])
    x2 = max(dict["x"])
    y1 = min(dict["y"])
    y2 = max(dict["y"])
    box_image = image[y1:y2, x1:x2]
    box_image = cv2.cvtColor(box_image, cv2.COLOR_BGR2GRAY)
    box_image = cv2.resize(box_image, (128, 128))
    X.append(box_image)
    y.append(dict["data"])

X = np.array(X)
y = np.array(y)

label_encoder = loaded_label_encoder
y_encoded = label_encoder.fit_transform(y)
y_onehot = to_categorical(y_encoded, num_classes=len(label_encoder.classes_))


# 데이터 확장을 위한 이미지 변환 함수 정의
import cv2
import random
import numpy as np

def augment_image(image):
    # 이미지 변환 방식 무작위 선택
    transform_type = random.choice(["rotate", "brightness", "color", "stretch", "noise"])

    if transform_type == "rotate":
        angle = random.randint(-30, 30)
        rows, cols = image.shape
        rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        transformed_image = cv2.warpAffine(image, rotation_matrix, (cols, rows))
    elif transform_type == "brightness":
        brightness_factor = random.uniform(0.7, 1.3)
        transformed_image = cv2.convertScaleAbs(image, alpha=brightness_factor, beta=0)
    elif transform_type == "color":
        color_factor = random.uniform(0.5, 1.5)
        transformed_image = image * color_factor
        transformed_image = np.clip(transformed_image, 0, 255).astype(np.uint8)
    elif transform_type == "stretch":
        stretch_factor_x = random.uniform(0.8, 1.2)
        stretch_factor_y = random.uniform(0.8, 1.2)
        rows, cols = image.shape
        stretched_image = cv2.resize(image, (int(cols * stretch_factor_x), int(rows * stretch_factor_y)))
        transformed_image = cv2.resize(stretched_image, (128, 128))
    elif transform_type == "noise":
        gaussian_noise = np.zeros((image.shape[0], image.shape[1], 1), dtype=np.uint8)
        cv2.randn(gaussian_noise, 128, 12)
        noisy_image = cv2.add(image, gaussian_noise)
        transformed_image = np.clip(noisy_image, 0, 255).astype(np.uint8)

    return transformed_image



augmented_X = []
augmented_y_onehot = []

for i in range(X.shape[0]):
    original_image = X[i]
    augmented_X.append(original_image)
    augmented_y_onehot.append(y_onehot[i])

    # 이미지 확장을 위해 변환된 이미지 추가
    for _ in range(5):  # 30개의 변환된 이미지 추가
        transformed_image = augment_image(original_image)
        sized_transformed_image = cv2.resize(transformed_image, (128, 128))
        augmented_X.append(sized_transformed_image)
        augmented_y_onehot.append(y_onehot[i])

# 데이터 확장 후 배열로 변환
augmented_X = np.array(augmented_X)
augmented_y_onehot = np.array(augmented_y_onehot)

# 훈련 및 검증 데이터 분할
X_train, X_val, y_train, y_val = train_test_split(augmented_X, augmented_y_onehot, test_size=0.2, random_state=42)


model = loaded_model

# 모델 훈련
model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val), callbacks=[early_stopping])

# 새로운 이미지 예측 함수 정의
def predict_new_image(image_path, model, label_encoder):
    new_image = load_and_preprocess_image(image_path)
    prediction = model.predict(np.array([new_image]))
    predicted_class = np.argmax(prediction)
    predicted_label = label_encoder.inverse_transform([predicted_class])[0]
    return predicted_label

# 새로운 이미지 예측
new_image_path = 'C:/upload/image3.png'
predicted_label = predict_new_image(new_image_path, model, label_encoder)
print("예측된 레이블:", predicted_label)

Epoch 1/5
9/9 [==============================] - 4s 370ms/step - loss: 2.2283 - accuracy: 0.5755 - val_loss: 3.5844 - val_accuracy: 0.0714
Epoch 2/5
9/9 [==============================] - 3s 336ms/step - loss: 1.8630 - accuracy: 0.6259 - val_loss: 3.2590 - val_accuracy: 0.2286
Epoch 3/5
9/9 [==============================] - 3s 339ms/step - loss: 1.4370 - accuracy: 0.7050 - val_loss: 2.9403 - val_accuracy: 0.3429
Epoch 4/5
9/9 [==============================] - 3s 340ms/step - loss: 1.1616 - accuracy: 0.7626 - val_loss: 2.6648 - val_accuracy: 0.4429
Epoch 5/5
1/1 [==============================] - 0s 81ms/step
예측된 레이블: 유앙페


In [18]:
# 모델 저장
model_filepath = 'C:/upload/new_handwriting_model.h5'
label_encoder_filepath = 'C:/upload/new_handwriting_label_encoder.pkl'

import pickle
# 모델 저장 함수 정의
def save_model(model, model_filepath, label_encoder_filepath):
    model.save(model_filepath)

    with open(label_encoder_filepath, 'wb') as f:
        pickle.dump(label_encoder, f)

        
save_model(model, model_filepath, label_encoder_filepath)

# XXXXXXXXXXXXXXXXXXXXXX 이전 버전XXXXXXXXXXXXXXXXXXXXX

In [ ]:
model = keras.models.load_model(r"C:/upload/handwriting_model.h5")

In [ ]:
model.save(r"C:/upload/handwriting_model.h5")

In [48]:
import json

# JSON 파일 경로
json_file_path = 'C:/upload/IMG_OCR_53_4PO_09451.json'

# JSON 파일을 읽어와서 지정된 범위의 데이터를 추출
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    json_lines = json_file.readlines()[29:-1]  # 30번째 줄부터 마지막 줄의 이전 줄까지

# 개행 문자를 제거하고 "bbox" 부분 제거하여 JSON 데이터로 파싱
json_data = ''.join(line.strip().replace('"bbox": ', '') for line in json_lines)
data = json.loads(json_data)
data

[{'data': '09451',
  'id': 1,
  'x': [1848, 1848, 2019, 2019],
  'y': [199, 275, 199, 275]},
 {'data': '먹방', 'id': 2, 'x': [812, 812, 938, 938], 'y': [653, 726, 653, 726]},
 {'data': '공맹수',
  'id': 3,
  'x': [717, 717, 897, 897],
  'y': [832, 932, 832, 932]},
 {'data': '060',
  'id': 4,
  'x': [1729, 1729, 1821, 1821],
  'y': [769, 838, 769, 838]},
 {'data': '8978',
  'id': 5,
  'x': [1847, 1847, 1977, 1977],
  'y': [776, 837, 776, 837]},
 {'data': '1426',
  'id': 6,
  'x': [2008, 2008, 2131, 2131],
  'y': [770, 836, 770, 836]},
 {'data': '050',
  'id': 7,
  'x': [1730, 1730, 1816, 1816],
  'y': [864, 928, 864, 928]},
 {'data': '7156',
  'id': 8,
  'x': [1838, 1838, 1948, 1948],
  'y': [868, 925, 868, 925]},
 {'data': '3106',
  'id': 9,
  'x': [1967, 1967, 2083, 2083],
  'y': [870, 931, 870, 931]},
 {'data': '워라밸',
  'id': 10,
  'x': [749, 749, 919, 919],
  'y': [1163, 1251, 1163, 1251]},
 {'data': '030',
  'id': 11,
  'x': [1199, 1199, 1275, 1275],
  'y': [1211, 1271, 1211, 1271]},
 {

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

# 원래 한글 문장
original_text = "안녕하세요, 한글 tokenizer 인코딩 예제입니다."

# tokenizer 객체 생성
tokenizer = Tokenizer(char_level=True)

# tokenizer를 학습하여 문장을 숫자로 인코딩
tokenizer.fit_on_texts([original_text])

# 인코딩된 데이터 생성
encoded_data = tokenizer.texts_to_sequences([original_text])[0]

# 인코딩된 데이터 출력
print(encoded_data)


In [ ]:
# 인덱스를 문자로 변환하여 디코딩된 문장 생성
decoded_text = tokenizer.sequences_to_texts([encoded_data])[0]

# 디코딩된 문장 출력
print(decoded_text)


In [ ]:
import tensorflow as tf
import json
from tensorflow.keras.preprocessing.text import Tokenizer

# JSON 파일 경로
json_file_path = 'C:/upload/IMG_OCR_53_4PO_09451.json'

# JSON 파일을 읽어와서 지정된 범위의 데이터를 추출
with open(json_file_path, 'r', encoding='utf-8') as json_file:
    json_lines = json_file.readlines()[29:-1]  # 30번째 줄부터 마지막 줄의 이전 줄까지

# 개행 문자를 제거하고 "bbox" 부분 제거하여 JSON 데이터로 파싱
source_json_data = ''.join(line.strip().replace('"bbox": ', '') for line in json_lines)
json_data = json.loads(source_json_data)

# Tokenizer 생성 및 데이터 학습
tokenizer = Tokenizer(char_level=True)
data = [item["data"] for item in json_data]
tokenizer.fit_on_texts(data)

# 데이터 인코딩
new_data = []
encoded_data_list = []
for item in json_data:
    encoded_data = tokenizer.texts_to_sequences([item["data"]])[0]
    new_item = {
        "data": encoded_data,
        "id": item["id"],
        "x": item["x"],
        "y": item["y"]
    }
    new_data.append(new_item)

# 결과 출력 (예시로 처음 5개 데이터만 출력)
for idx, item in enumerate(new_data[:5]):
    print(f"Data {idx + 1}:")
    print("  'data':", item["data"])
    print("  'id':", item["id"])
    print("  'x':", item["x"])
    print("  'y':", item["y"])


In [ ]:
# 디코딩 함수 정의
def decode_sequence(encoded_sequence):
    return tokenizer.sequences_to_texts([encoded_sequence])[0]

# 결과 출력 및 디코딩 (예시로 처음 5개 데이터만 출력)
for idx, item in enumerate(new_data[:5]):
    print(f"Data {idx + 1}:")
    print("  'data':", decode_sequence(item["data"]))
    print("  'id':", item["id"])
    print("  'x':", item["x"])
    print("  'y':", item["y"])

In [ ]:
data

In [ ]:
new_data